## Ensembling

In [1]:
from lightning import pytorch as pl
import numpy as np
import torch
from chemprop import data, models, nn

This is an example [dataloader](./data/dataloaders.ipynb).

In [2]:
smis = ["C" * i for i in range(1, 4)]
ys = np.random.rand(len(smis), 1)
dset = data.MoleculeDataset([data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(smis, ys)])
dataloader = data.build_dataloader(dset)

### Model ensembling

A single model will sometimes give erroneous predictions for some molecules. These erroneous predictions can be mitigated by averaging the predictions of several models trained on the same data. 

In [3]:
ensemble = []
n_models = 3
for _ in range(n_models):
    ensemble.append(models.MPNN(nn.BondMessagePassing(), nn.MeanAggregation(), nn.RegressionFFN()))

In [4]:
for model in ensemble:
    trainer = pl.Trainer(logger=False, enable_checkpointing=False, max_epochs=1)
    trainer.fit(model, dataloader)

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
/home/scli/miniconda3/envs/chemprop_v2/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/scli/miniconda3/envs/chemprop_v2/lib/python3.1 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/scli/miniconda3/envs/chemprop_v2/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is t

Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  2.67it/s, train_loss=0.282]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  2.67it/s, train_loss=0.282]

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name            | Type               | Params
-------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K 
1 | agg             | MeanAggregation    | 0     
2 | bn              | Identity           | 0     
3 | predictor       | RegressionFFN      | 90.6 K
4 | X_d_transform   | Identity           | 0     
5 | metrics         | ModuleList         | 0     
--------------------------------------------------

...

Epoch 0: 100%|██████████| 1/1 [00:00<00:00, 153.04it/s, train_loss=0.230]

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name            | Type               | Params
-------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K 
1 | agg             | MeanAggregation    | 0     
2 | bn              | Identity           | 0     
3 | predictor       | RegressionFFN      | 90.6 K
4 | X_d_transform   | Identity           | 0     
5 | metrics         | ModuleList         | 0     
--------------------------------------------------


Epoch 0: 100%|██████████| 1/1 [00:00<00:00, 184.54it/s, train_loss=0.274]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:00<00:00, 167.10it/s, train_loss=0.274]


In [5]:
prediction_dataloader = data.build_dataloader(dset, shuffle=False)
predictions = []
for model in ensemble:
    predictions.append(torch.concat(trainer.predict(model, prediction_dataloader)))

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/scli/miniconda3/envs/chemprop_v2/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 542.25it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 585.80it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 582.38it/s]


In [6]:
predictions

[tensor([[0.0236],
         [0.0247],
         [0.0272]]),
 tensor([[0.0671],
         [0.0791],
         [0.0826]]),
 tensor([[0.0140],
         [0.0346],
         [0.0442]])]

In [7]:
torch.concat(predictions, axis=1).mean(axis=1, keepdim=True)

tensor([[0.0349],
        [0.0461],
        [0.0514]])